<a href="https://colab.research.google.com/github/meyush0/Text-Classification/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier

import re
from nltk.stem.porter import PorterStemmer

In [ ]:
data = pd.read_csv('/content/augmented_dataset_1000.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data['Feedback'][526]

In [ ]:
#cleaning the Feedback
ps=PorterStemmer()
corpus=[]
for i in range(0,len(data)):
    print(i)
    hmm=re.sub('[^a-zA-Z]',"  ",data['Feedback'][i])
    hmm=hmm.lower()
    hmm=hmm.split()


    hmm = ' '.join([ps.stem(words) for words in hmm if not words in stopwords.words('english')])


    corpus.append(hmm)

In [ ]:
corpus

In [ ]:
corpus[6555]

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

One hot layer

In [ ]:
voc_size=20000

In [ ]:
one_hot=[one_hot(words,voc_size) for words in corpus]
one_hot


In [ ]:
#one_hot_implementation = [one_hot(' '.join(feedback), voc_size) for feedback in data['Feedback']]
#print(one_hot_implementation)

In [ ]:
sent_len=60
embeded_sent=pad_sequences(one_hot,padding='pre',maxlen=sent_len)

In [ ]:
embeded_sent

In [ ]:
embeded_sent.size

In [ ]:
data['Theme']

In [ ]:
data['Theme'].value_counts()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categorical_columns = ['Theme']
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(data[categorical_columns])
encoded_features_array = encoded_features.toarray()

encoded_features_df=pd.DataFrame(encoded_features_array,columns=encoder.get_feature_names_out(categorical_columns))
X=pd.concat([data, encoded_features_df],axis=1)

In [ ]:
X

In [ ]:
theme_mapping = {
    'Assessment, Marking and Feedback': 0,
    'Enrolment, Induction and Academic Support': 1,
    'Teaching on my Course': 2,
    'Organisation and Management': 3,
    'Student Voice': 4,
    'Other': 5,
    'Learning Resources, Opportunities and Community': 6
}

In [ ]:
data['Numerical_Theme'] = data['Theme'].map(theme_mapping)
print(data[['Theme', 'Numerical_Theme']])

Splitting the data

In [ ]:
y=data['Numerical_Theme']
y

In [ ]:
x=embeded_sent
x

In [ ]:
#training the model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x
                                                    , y
                                                    , test_size=0.2,
                                                    random_state=25)
x_train_processed = [' '.join(map(str, array)) for array in x_train]
x_test_processed = [' '.join(map(str, array)) for array in x_test]

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

NaiveBayes/

In [ ]:
model = make_pipeline(
    TfidfVectorizer(),
    MultinomialNB()
)
model.fit(x_train_processed,y_train)

In [ ]:
y_pred=model.predict(x_test_processed)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(y_test, y_pred)

# heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(1, 8), yticklabels=range(1, 8))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'tfidfvectorizer__max_features': [5000, 10000, 20000],
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
    'multinomialnb__alpha': [0.1, 0.5, 1.0]
}

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_processed, y_train)

In [ ]:
print("Best Parameters:", grid_search.best_params_)

In [ ]:
print("Best Accuracy:", grid_search.best_score_ * 100)

Random Forest Classifier

In [ ]:
model2 = make_pipeline(
     TfidfVectorizer(),
     XGBClassifier()
)

In [ ]:
model2.fit(x_train_processed,y_train)

In [ ]:
y_pred=model2.predict(x_test_processed)
y_pred

In [ ]:
#accuracy = accuracy_score(y_test, y_pred)
#prinblend_test_set = np.column_stack((pred_nb_test, pred_svm_test))(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
param_grid_rf = {
    'tfidfvectorizer__max_features': [5000, 10000, 20000],
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__max_depth': [None, 10, 20],
    'randomforestclassifier__min_samples_split': [2, 5, 10]
}

In [ ]:
grid_search_rf = GridSearchCV(model2, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
# voting classifier # we can use multiple algo in the pipeline
#ensemble_model = make_pipeline(
       # PreprocessingStep(),
       # TfidfVectorizer(),
       # FunctionTransformer(lambda x: x.toarray(), accept_sparse=True),
# SomeAlgo())

impleminting VotingClassifier

In [ ]:
classifier1 = RandomForestClassifier(random_state=42)
classifier2 = GradientBoostingClassifier(random_state=42)
classifier3 = LogisticRegression(random_state=42)

In [ ]:
classifier_hard = VotingClassifier(
    estimators=[('rf',RandomForestClassifier(random_state=42)), ('gb', GradientBoostingClassifier(random_state=42)), ('lr', LogisticRegression(random_state=42))],
    voting='hard')

In [ ]:
classifier_soft = VotingClassifier(
    estimators=[('rf',RandomForestClassifier(random_state=42)), ('gb', GradientBoostingClassifier(random_state=42)), ('lr', LogisticRegression(random_state=42))],
    voting='soft')

Impleminting ANN

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU,LeakyReLU,ELU,PReLU
from tensorflow.keras.layers import Dropout

In [ ]:
yep=tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.2,verbose=0,baseline=None,restore_best_weights=True,
                                             start_from_epoch=0,patience=2)     #applying earlystopping

Model Initiate

In [ ]:
model=Sequential()
model.add(Dense(units=10,activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)  #adding optimizer

In [ ]:
model.compile(optimizer=opt,loss='mean_squared_error',metrics=['mean_absolute_error'])    #complir the model

In [ ]:
#fitting the model
ANN=model.fit(x_train, y_train,validation_split=0.2,batch_size=50,epochs=70)

In [ ]:
model.summary()

In [ ]:
ANN.history.keys()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred

Using Blending Ensemble learning technique

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Train the first base model (Multinomial Naive Bayes)
model_nb = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_nb.fit(x_train, y_train)

In [ ]:
# Train the second base model (Support Vector Machine)
model_svm = make_pipeline(TfidfVectorizer(), SVC(probability=True))
model_svm.fit(x_train, y_train)

In [ ]:
pred_nb = model_nb.predict(x_blend)   #First base model prediction

In [ ]:
pred_svm = model_svm.predict(x_blend)  #predict from second base model

In [ ]:
# Create a blend set for the meta-model
blend_set = np.column_stack((pred_nb, pred_svm))
print(blend_set)

In [ ]:
# Train the meta-model (Random Forest)
meta_model = RandomForestClassifier()
meta_model.fit(blend_set, y_blend)

In [ ]:
pred_nb_test = model_nb.predict(x_test_processed)
pred_svm_test = model_svm.predict(x_test_processed)

In [ ]:
blend_test_set = np.column_stack((pred_nb_test, pred_svm_test))
final_predictions = meta_model.predict(blend_test_set)

In [ ]:
# Evaluate the final ensemble model
ensemble_accuracy = accuracy_score(y_test, final_predictions)
ensemble_accuracy*100

Aplying more RNN model of Deep learning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from sklearn.base import BaseEstimator, TransformerMixin   # framework for creating  custom model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
class LSTMpipeline(BaseEstimator , TransformerMixin):
  def __init__(self, voc_size=20000 , max_len=60 , lstm_units=64 , epochs=7 , batch_size=32 , dropout=0.2):
    self.voc_size = voc_size
    self.max_len = max_len
    self.lstm_units = lstm_units
    self.dropout = dropout
    self.epochs = epochs
    self.batch_size = batch_size
    self.model = None
    self.tokenizer = None

  def fit(self, x, y):

    tokenizer = Tokenizer(num_words=self.voc_size, oov_token="<OOV>")
    tokenizer.fit_on_texts(x)
    x_sequences = tokenizer.texts_to_sequences(x)
    x_padded = pad_sequences(x_sequences, maxlen=self.max_len, padding='pre')

    self.model = Sequential()
    self.model.add(Embedding(self.voc_size, 128, input_length=self.max_len))
    self.model.add(LSTM(self.lstm_units, dropout=self.dropout))
    self.model.add(Dense(len(set(y)), activation='softmax'))

    self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    self.model.fit(x_padded, y, epochs=self.epochs, batch_size=self.batch_size)

    return self

  def transform(self, x):
    x_sequences = tokenizer.texts_to_sequences(x)
    X_padded = pad_sequences(x_sequences, maxlen=self.max_len, padding='pre')
    return x_padded


In [ ]:
def predict(self, x):
        x_transformed = self.transform(x)
        y_pred = self.model.predict_classes(x_transformed)
        return y_pred

In [ ]:
lstm_pipeline = Pipeline([
    ('lstm', LSTMpipeline())
])

In [ ]:
lstm_pipeline.fit(x_train,y_train)

for augmented_data_set_1500

In [ ]:
data2 = pd.read_csv('/content/augmented_dataset_1500.csv')
data2.head()

In [ ]:
def preprocess_and_split(data, text_column, label_column, voc_size=20000, sent_len=60, test_size=0.2, random_state=42):
    # Clean the text data
    corpus = []
    for i in range(len(data)):
        hmm = re.sub('[^a-zA-Z]', " ", data[text_column][i])
        hmm = hmm.lower()
        hmm = hmm.split()
        hmm = ' '.join([ps.stem(words) for words in hmm if not words in stopwords.words('english')])
        corpus.append(hmm)

    # One-hot encode the text data
    one_hot_encoded_list = [one_hot(words, voc_size) for words in corpus]

    # Pad sequences for uniform length
    embedded_sentences = pad_sequences(one_hot_encoded_list, padding='pre', maxlen=sent_len)

    # Encode the labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(data[label_column])

    # Split the data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(embedded_sentences, encoded_labels, test_size=test_size, random_state=random_state)

    return x_train, x_test, y_train, y_test


In [ ]:
class LSTMpipeline(BaseEstimator, TransformerMixin):
    def __init__(self, voc_size=20000, max_len=60, lstm_units=64, epochs=7, batch_size=32, dropout=0.2):
        self.voc_size = voc_size
        self.max_len = max_len
        self.lstm_units = lstm_units
        self.dropout = dropout
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.tokenizer = None

    def fit(self, x, y):
        self.tokenizer = Tokenizer(num_words=self.voc_size, oov_token="<OOV>")
        self.tokenizer.fit_on_texts(x)
        x_sequences = self.tokenizer.texts_to_sequences(x)
        x_padded = pad_sequences(x_sequences, maxlen=self.max_len, padding='pre')

        num_classes = len(set(y))

        self.model = Sequential()
        self.model.add(Embedding(self.voc_size, 128, input_length=self.max_len))
        self.model.add(LSTM(self.lstm_units, dropout=self.dropout))
        self.model.add(Dense(num_classes, activation='softmax'))

        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model.fit(x_padded, y, epochs=self.epochs, batch_size=self.batch_size)

        return self

    def transform(self, x):
      x_sequences = self.tokenizer.texts_to_sequences(x)
      x_padded = pad_sequences(x_sequences, maxlen=self.max_len, padding='pre')
      return x_padded

In [ ]:
lstm_pipeline = Pipeline([
    ('lstm', LSTMpipeline())
])

In [ ]:
lstm_pipeline.fit(x_train,y_train)